In [ ]:
#!pip install curl_cffi
import functions
import matplotlib.pyplot as plt
import pandas as pd
import plotly as pl
import seaborn as sns

url = 'data/combined_job_offers.csv'
url2 = 'data/combined_job_offers3.csv'
url_scraped = 'data/scraped_data.csv'

# reading the 2 base datasets
df1 = pd.read_csv(url)
df2 = pd.read_csv(url2)

df3 = pd.concat([df1,df2],axis=0)

#reading the dataset, scraped from stepstone
df_scraped = pd.read_csv(url_scraped)

pd.set_option('display.max_columns', None)

In [ ]:
# cleaning the base dataset
# 1 drop columns df3
df3.drop(columns =['repost_date', 'email', 'job_desc'], inplace=True)

# 2 Renaming 'link'
df3.rename(columns={'link': 'source'}, inplace=True)

# 3 replace all links with LinkedIn
# apply lambda for each cell replace all string
df3['source'] = df3['source'].apply(lambda x: 'LinkedIn')

In [ ]:
# remove duplicates from scraped dataset
df_scraped.drop_duplicates(subset=['job_title', 'company_name', 'post_date'], keep='last', inplace=True)

# combine scraped dataset with base dataset
df_combined = pd.concat([df3,df_scraped],axis=0)

# apply wrangling the job_levels
df_combined['job_level'] = df_combined.apply(replace_nan_with_job_level, axis=1)
df_combined['sector'] = df_combined.apply(replace_sectors, axis=1)

In [ ]:
top_5_sectors = test['sector'].value_counts().head(10)

In [ ]:
test.groupby(["job_level", "sector"])["job_type"].count()

In [ ]:
# job categories and experience
job_experience = {
    'Entry level': 0-3,
    'Junior': 1,  # Averaging between 0-2 years
    'Associate': 2,  # Averaging between 1-3 years
    'Mid-Senior level': 4,  # Averaging between 3-5 years
    'Senior': 7.5,  # Averaging between 5-10 years
    'Lead': 7.5,  # Averaging between 5-10 years
    'Manager': 10,  # At the start of the 5-10+ range
    'Executive': 10,
    'Director': 10,
    'Consultant': 10,
    'Unknown': float('nan')  # For visualization purposes, use NaN
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(job_experience.items()), columns=['Sector', 'Years of Experience'])

# Exclude 'Unknown' for some visualizations if they include NaN
df_filtered = df.dropna()

# Plotting with Seaborn
plt.figure(figsize=(6, 3))
sns.barplot(x='Sector', y='Years of Experience', data=df_filtered, palette='viridis')

plt.title('Breakdown of Job Categories by Years of Experience')
plt.xlabel('Job Sector')
plt.ylabel('Years of Experience')
plt.xticks(rotation=60)
plt.show()

In [ ]:

grouped_data = test.groupby(["job_level", "sector"])["job_type"].count().reset_index()

# Rename the count column for clarity
grouped_data = grouped_data.rename(columns={"job_type": "type_count"})

# Create a bar plot using Seaborn
plt.figure(figsize=(10, 6))
sns.barplot(data=grouped_data, x="job_level", y="type_count", hue="sector", dodge=True)

# Add titles and labels
plt.title("Job Type Count by Job Level and Sector")
plt.xlabel("Job Level")
plt.ylabel("Count of Job Types")
plt.legend(title="Sector")
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
# Define job sectors and hypothetical counts of data analysts
job_sectors = {
    'Information Technology & Services': 700,
    'Financial Services': 500,
    'Engineering Services': 400,
    'Software Development': 650,
    'Retail': 300,
    'Marketing Services': 350,
    'Telecommunications': 450,
    'Biotechnology': 200,
    'Education': 250,
    'Healthcare': 300
}

# Convert to DataFrame
df_sectors = pd.DataFrame(list(job_sectors.items()), columns=['Sector', 'Data Analysts'])

# Sort the DataFrame for a better visualization
df_sectors = df_sectors.sort_values(by='Data Analysts', ascending=False)

# Plot using Seaborn, flipping x and y
plt.figure(figsize=(6, 4))
sns.barplot(y='Data Analysts', x='Sector', data=df_sectors, palette='coolwarm')

plt.title('Top 10 Job Sectors for Data Analysts in Germany')
plt.ylabel('Number of Data Analysts')
plt.xlabel('Job Sector')
plt.xticks(rotation=45)  # Rotate x labels for better visibility
plt.show()